buat database

In [1]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    Float,
    insert,
    inspect,
    text,
)

engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

# create city SQL table
table_name = "receipts"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("customer_name", String(16), primary_key=True),
    Column("price", Float),
    Column("tip", Float),
)
metadata_obj.create_all(engine)

buat data dummy

In [2]:
rows = [
    {"receipt_id": 1, "customer_name": "Alan Payne", "price": 12.06, "tip": 1.20},
    {"receipt_id": 2, "customer_name": "Alex Mason", "price": 23.86, "tip": 0.24},
    {"receipt_id": 3, "customer_name": "Woodrow Wilson", "price": 53.43, "tip": 5.43},
    {"receipt_id": 4, "customer_name": "Margaret James", "price": 21.11, "tip": 1.00},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

query sederhana

In [3]:
with engine.connect() as con:
    rows = con.execute(text("""SELECT * from receipts"""))
    for row in rows:
        print(row)

(1, 'Alan Payne', 12.06, 1.2)
(2, 'Alex Mason', 23.86, 0.24)
(3, 'Woodrow Wilson', 53.43, 5.43)
(4, 'Margaret James', 21.11, 1.0)


struktur database

In [4]:
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
print(table_description)

Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT


buat tool yang akan terkoneksi ke database

In [5]:
from transformers.agents import tool


@tool
def sql_engine(query: str) -> str:
    """
    Allows you to perform SQL queries on the table. Returns a string representation of the result.
    The table is named 'receipts'. Its description is as follows:
        Columns:
        - receipt_id: INTEGER
        - customer_name: VARCHAR(16)
        - price: FLOAT
        - tip: FLOAT

    Args:
        query: The query to perform. This should be correct SQL.
    """
    output = ""
    with engine.connect() as con:
        rows = con.execute(text(query))
        for row in rows:
            output += "\n" + str(row)
    return output

buat agent (model)

In [6]:
from huggingface_hub import login
login(token="token huggingface")

In [7]:
from transformers.agents import ReactCodeAgent, HfApiEngine

agent = ReactCodeAgent(
    tools=[sql_engine],
    llm_engine=HfApiEngine("meta-llama/Meta-Llama-3-8B-Instruct"),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

jalankan prompt

In [8]:
agent.run("Can you give me the name of the client who got the most expensive receipt?")

======== New task ========
Can you give me the name of the client who got the most expensive receipt?
=== Agent thoughts:
Thought: To solve this task, I will use the `sql_engine` tool to query the'receipts' table and get the receipt with the highest price. Then, I will use the `final_answer` tool to provide the name of the client.
>>> Agent is executing the code below:
query = "SELECT customer_name, MAX(price) FROM receipts GROUP BY customer_name"
result = sql_engine(query)
print(result)
====
Print outputs:

('Alan Payne', 12.06)
('Alex Mason', 23.86)
('Margaret James', 21.11)
('Woodrow Wilson', 53.43)

=== Agent thoughts:
Thought: From the previous step's output, I can see that the receipts with the highest price are from 'Woodrow Wilson'. To get the name of the client who got the most expensive receipt, I will extract the client name from the output.
>>> Agent is executing the code below:
client_name = result[0][0]
final_answer(client_name)
====
Print outputs:

Last output from code 

'\n'

lebih advanced

buat tabel yang perlu join

In [7]:
table_name = "waiters"
receipts = Table(
    table_name,
    metadata_obj,
    Column("receipt_id", Integer, primary_key=True),
    Column("waiter_name", String(16), primary_key=True),
)
metadata_obj.create_all(engine)

rows = [
    {"receipt_id": 1, "waiter_name": "Corey Johnson"},
    {"receipt_id": 2, "waiter_name": "Michael Watts"},
    {"receipt_id": 3, "waiter_name": "Michael Watts"},
    {"receipt_id": 4, "waiter_name": "Margaret James"},
]
for row in rows:
    stmt = insert(receipts).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

struktur database

In [8]:
updated_description = """Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:"""

inspector = inspect(engine)
for table in ["receipts", "waiters"]:
    columns_info = [(col["name"], col["type"]) for col in inspector.get_columns(table)]

    table_description = f"Table '{table}':\n"

    table_description += "Columns:\n" + "\n".join([f"  - {name}: {col_type}" for name, col_type in columns_info])
    updated_description += "\n\n" + table_description

print(updated_description)

Allows you to perform SQL queries on the table. Beware that this tool's output is a string representation of the execution output.
It can use the following tables:

Table 'receipts':
Columns:
  - receipt_id: INTEGER
  - customer_name: VARCHAR(16)
  - price: FLOAT
  - tip: FLOAT

Table 'waiters':
Columns:
  - receipt_id: INTEGER
  - waiter_name: VARCHAR(16)


gunakan agent yang lebih powerful yaitu qwen

In [10]:
sql_engine.description = updated_description

from transformers.agents import ReactCodeAgent, HfApiEngine

agent = ReactCodeAgent(
    tools=[sql_engine],
    llm_engine=HfApiEngine("Qwen/Qwen2.5-72B-Instruct"),
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

jalankan prompt

In [11]:
agent.run("Which waiter got more total money from tips?")

======== New task ========
Which waiter got more total money from tips?
=== Agent thoughts:
Thought: To find out which waiter got more total money from tips, I will perform a SQLquery to calculate thetotalamountoftips foreachwaiter and then returnthe waiter with thehighesttotal.
>>> Agent is executing the code below:
sql_query = "SELECT waiter_name, SUM(tip) as total_t FROM waitersJOIN receiptsONwaiters.receipt_id = receipts.receipt_id GROUP BYwaiter name ORDERBYtotal t DESC LIMIT 1"
result =Sql(query=sql_query)
print(result)
====
Code execution failed due to the following error:
EXECUTION FAILED:
Evaluation stopped at line 'result =Sql(query=sql_query)' because of the following error:
It is not permitted to evaluate other functions than the provided tools or functions defined in previous code (tried to execute Sql).
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/agents/python_interpreter.py", line 902, in evaluate_python_code
    result

'To determine which waiter received the most total money from tips, I will execute a SQL query to sum the tips for each waiter and return the waiter with the highest total. Here is the corrected code to achieve that:\n\n```py\nsql_query = """\nSELECT waiter_name, SUM(tip) as total_tips\nFROM waiters\nJOIN receipts ON waiters.receipt_id = receipts.receipt_id\nGROUP BY waiter_name\nORDER BY total_tips DESC\nLIMIT 1\n"""\nresult = sql(query=sql_query)\nprint(result)\n```\n\nIf you run this code, it should provide the name of the waiter who received the highest total amount in tips. If there are any issues with running the code, I can help further.'

Terkadang agent/model salah memberikan query yang menyebabkan error

SUMBER 

https://huggingface.co/learn/cookbook/agent_text_to_sql